In [13]:
#Parsing STiX 1.x Data and creating a common format
import xml.etree.ElementTree as ET
tree = ET.parse("./dataset/stix1_sample.xml")
modified_dict = {
    "stix_version": "",
    "entity_type": "",
    "indicator_type": "",
    "description": "",
    "timestamp":"",
    "values": None,
    "validation" : False
}
stix1_schema = {

 "STIX_Package": [
     "version",
     "id",
     "timestamp",
     "Indicators",
     "Incidents",
     "Campaigns",
     "Threat_Actors",
 ],

 "Indicator": [
     "id",
     "timestamp",
     "Type",
     "Description",
     "Observable",
 ],

 "Observable": [
     "id",
     "Object",
     "Title",
     "Description"
 ],

 "Object": [
     "Properties"
 ],

 "URIObject": [
     "Value",
     "condition",
     "apply_condition",
     "type"
 ],

 "Incident": [
     "id",
     "timestamp",
     "Description",
     "Time"
 ],

 "TTP": [
     "id",
     "Title",
     "Description",
 ],

 "Campaign": [
     "id",
     "Title",
     "Description",
 ],

 "ThreatActor": [
     "id",
     "Title",
     "Description",
 ]
}

valid_tags = []
valid_attributes = []
def strip_ns(tag):
    return tag.split("}")[-1]
root = tree.getroot()
version = root.get("version")
for inx in root.iter():
    clean = strip_ns(inx.tag)
    valid_tags.append(clean)
    attr = []
    for a in inx.attrib.keys():
        valid_attributes.append(a)
tags_val = False
attr_val = False
for key, attributes in stix1_schema.items():
    if key in valid_tags:
        tags_val = True
        for at in attributes:
            if at in valid_attributes:
               attr_val = True
            else:
                attr_val = False
    else:
        continue
if tags_val == True and attr_val == True:
    modified_dict["validation"] = True
else:
    modified_dict["validation"] = False
modified_dict["stix_version"]=version
ns = {
    "stix": "http://docs.oasis-open.org/cti/ns/stix/core-1",
    "indicator": "http://docs.oasis-open.org/cti/ns/stix/indicator-1",
    "URIObject": "http://docs.oasis-open.org/cti/ns/cybox/objects/uri-2"
}
entity_type_array = ["Indicator", "Incident", "TTP", "Campaign", "ThreatActor"]
for elem in root.iter():
    clean = strip_ns(elem.tag)
    if clean in entity_type_array:
        modified_dict["entity_type"]=clean
mod_array = [".//indicator:Type", ".//indicator:Description", ".//URIObject:Value"]
new = [(root.find(i, ns).text if root.find(i, ns) is not None else "") for i in mod_array]
modified_dict["indicator_type"]=new[0]
modified_dict["description"]=new[1]
modified_dict["values"]=new[2].split("##comma##")
indicator = root.find(".//stix:Indicator", ns)
modified_dict["timestamp"] = indicator.get("timestamp") or ""
print(modified_dict)

AttributeError: 'str' object has no attribute 'values'

In [40]:
#STiX 2.x Parsing
import json
with open("./dataset/stix20_sample.json") as f:
    item = json.load(f)
#print(len(item),"\n")
arr_of_data = []
def version_definition(data):
    try:
        item_out = list(data.keys())
        item_in = list(data["objects"][0].keys())
        merged_list = item_in + item_out
        print(merged_list)
        for indice in range(len(data["objects"])):
            modified_dict_20 = {
                "stix_version": "",
                "entity_type": "",
                "indicator_type": "",
                "description": "",
                "timestamp":"",
                "values": None,
                "validation": False
            }
            if "stix_version" in merged_list or "spec_version" in merged_list or "version" in merged_list:
                modified_dict_20["stix_version"] = 2.1
            else:
                modified_dict_20["stix_version"] = 2.0
            modified_dict_20["entity_type"] = item["objects"][indice]["type"] or "Couldn't find entity type"
            modified_dict_20["description"] = item["objects"][indice]["description"] or "Description not found"
            modified_dict_20["timestamp"] = item["objects"][indice]["created"] or None
            modified_dict_20["indicator_type"] = item["objects"][indice]["labels"] or "Indictor Type not found"
            object = item["objects"][indice]
            name = [object.get("name")] or ""
            alias = object.get("x_mitre_aliases") or []
            refs = str(object.get("external_references")) or []
            er = []
            for r in refs:
            if "external_id" in r:
                er.append(r["external_id"])
            elif "url" in r:
                er.append(r["url"])
            merged_values = name + alias + er
            modified_dict_20["values"] = merged_values
            arr_of_data.append(modified_dict_20)
    except Exception as e:
        print("Error accessing STiX data!", e)
version_definition(item)

['type', 'id', 'created', 'created_by_ref', 'revoked', 'external_references', 'object_marking_refs', 'modified', 'name', 'description', 'x_mitre_modified_by_ref', 'x_mitre_platforms', 'x_mitre_deprecated', 'x_mitre_domains', 'x_mitre_version', 'x_mitre_attack_spec_version', 'x_mitre_aliases', 'labels', 'type', 'id', 'objects']
